In [1]:
import pandas as pd
from anyio.lowlevel import cancel_shielded_checkpoint

from Data_Setup import init

In [2]:
# Initialize filter: starting stock & filter item code

last_stock_date = init.start_date - pd.Timedelta(days=1)
last_stock_date = last_stock_date.strftime('%Y-%m-%d')
item_code_filter = 'Item001'

In [3]:
# Read inbound & outbound data, include "Element_Type", filter out item code, and change the value of outbound amount to negative

inbound_data = pd.read_csv("Datasets/Inbound.csv")
inbound_data["Element_Type"] = "Purchase Order Item"
inbound_data = inbound_data[inbound_data['Item_Code'] == item_code_filter]

outbound_data = pd.read_csv("Datasets/Outbound.csv")
outbound_data['Amount'] = outbound_data['Amount'] * -1
outbound_data["Element_Type"] = "Sales Order Item"
outbound_data = outbound_data[outbound_data['Item_Code'] == item_code_filter]


In [4]:
merged_data = pd.concat([inbound_data, outbound_data], ignore_index=True)
merged_data.sort_values(by=['Date'], inplace=True, ignore_index=True)
merged_data[["Date","Element_Code","Element_Type","Warehouse_Code","Amount"]]


,Date,Element_Code,Element_Type,Warehouse_Code,Amount
0,2025-01-02,PO003-1,Purchase Order Item,WH002,570
1,2025-01-03,SO002-1,Sales Order Item,WH002,-490
2,2025-01-04,SO005-1,Sales Order Item,WH003,-520
3,2025-01-04,SO005-2,Sales Order Item,WH002,-870
4,2025-01-04,SO006-1,Sales Order Item,WH001,-50
5,2025-01-10,SO009-1,Sales Order Item,WH001,-650
6,2025-01-14,PO020-2,Purchase Order Item,WH001,800
7,2025-01-21,PO032-1,Purchase Order Item,WH002,640
8,2025-01-21,SO024-1,Sales Order Item,WH002,-770
9,2025-01-23,SO025-1,Sales Order Item,WH001,-740


In [5]:
available_stock = pd.read_csv("Datasets/Available_Stock.csv")
available_stock = available_stock[available_stock['Item_Code'] == item_code_filter]
available_stock = available_stock[available_stock['Date'] == last_stock_date]

available_stock


,Date,Warehouse_Code,Item_Code,Stock_Amount
0,2024-12-31,WH001,Item001,5800
15,2024-12-31,WH002,Item001,3100
30,2024-12-31,WH003,Item001,3200
45,2024-12-31,WH004,Item001,1600


In [6]:
columns = ['Date', 'Element_Code', 'Element_Type', 'Warehouse_Code']
for warehouse in init.warehouse_codes:
    columns.append(warehouse)

dashboard = pd.DataFrame(columns=columns)

first_row = [last_stock_date, None,'Available Stock', None, None, None, None, None]
dashboard.loc[0] = first_row

for index, row in available_stock.iterrows():
    WH = row['Warehouse_Code']
    available = row['Stock_Amount']
    dashboard.loc[0,WH] = available


dashboard

,Date,Element_Code,Element_Type,Warehouse_Code,WH001,WH002,WH003,WH004
0,2024-12-31,None,Available Stock,None,5800,3100,3200,1600


In [7]:
dashboard = dashboard.merge(right = merged_data, how='outer', on=['Date','Element_Code','Element_Type','Warehouse_Code'])
dashboard

,Date,Element_Code,Element_Type,Warehouse_Code,WH001,WH002,WH003,WH004,Element_Number,Item_Code,Amount
0,2024-12-31,None,Available Stock,None,5800,3100,3200,1600,NaN,NaN,NaN
1,2025-01-02,PO003-1,Purchase Order Item,WH002,NaN,NaN,NaN,NaN,PO003,Item001,570.0
2,2025-01-03,SO002-1,Sales Order Item,WH002,NaN,NaN,NaN,NaN,SO002,Item001,-490.0
3,2025-01-04,SO005-1,Sales Order Item,WH003,NaN,NaN,NaN,NaN,SO005,Item001,-520.0
4,2025-01-04,SO005-2,Sales Order Item,WH002,NaN,NaN,NaN,NaN,SO005,Item001,-870.0
5,2025-01-04,SO006-1,Sales Order Item,WH001,NaN,NaN,NaN,NaN,SO006,Item001,-50.0
6,2025-01-10,SO009-1,Sales Order Item,WH001,NaN,NaN,NaN,NaN,SO009,Item001,-650.0
7,2025-01-14,PO020-2,Purchase Order Item,WH001,NaN,NaN,NaN,NaN,PO020,Item001,800.0
8,2025-01-21,PO032-1,Purchase Order Item,WH002,NaN,NaN,NaN,NaN,PO032,Item001,640.0
9,2025-01-21,SO024-1,Sales Order Item,WH002,NaN,NaN,NaN,NaN,SO024,Item001,-770.0


In [8]:
dashboard = dashboard.drop(columns=['Element_Number', 'Item_Code'], errors='ignore')

cols = dashboard.columns.tolist()
cols.insert(4, cols.pop(cols.index('Amount')))
dashboard = dashboard[cols]
dashboard


,Date,Element_Code,Element_Type,Warehouse_Code,Amount,WH001,WH002,WH003,WH004
0,2024-12-31,None,Available Stock,None,NaN,5800,3100,3200,1600
1,2025-01-02,PO003-1,Purchase Order Item,WH002,570.0,NaN,NaN,NaN,NaN
2,2025-01-03,SO002-1,Sales Order Item,WH002,-490.0,NaN,NaN,NaN,NaN
3,2025-01-04,SO005-1,Sales Order Item,WH003,-520.0,NaN,NaN,NaN,NaN
4,2025-01-04,SO005-2,Sales Order Item,WH002,-870.0,NaN,NaN,NaN,NaN
5,2025-01-04,SO006-1,Sales Order Item,WH001,-50.0,NaN,NaN,NaN,NaN
6,2025-01-10,SO009-1,Sales Order Item,WH001,-650.0,NaN,NaN,NaN,NaN
7,2025-01-14,PO020-2,Purchase Order Item,WH001,800.0,NaN,NaN,NaN,NaN
8,2025-01-21,PO032-1,Purchase Order Item,WH002,640.0,NaN,NaN,NaN,NaN
9,2025-01-21,SO024-1,Sales Order Item,WH002,-770.0,NaN,NaN,NaN,NaN


In [9]:
for index, row in dashboard.iterrows():
    if index == 0:
        continue
    else:
        for col in dashboard.columns[5:]:
            if row['Warehouse_Code'] == col:
                dashboard.loc[index,col] = row['Amount'] + dashboard.loc[index-1,col]
            else:
                dashboard.loc[index,col] = dashboard.loc[index-1,col]

dashboard


,Date,Element_Code,Element_Type,Warehouse_Code,Amount,WH001,WH002,WH003,WH004
0,2024-12-31,None,Available Stock,None,NaN,5800,3100,3200,1600
1,2025-01-02,PO003-1,Purchase Order Item,WH002,570.0,5800,3670.0,3200,1600
2,2025-01-03,SO002-1,Sales Order Item,WH002,-490.0,5800,3180.0,3200,1600
3,2025-01-04,SO005-1,Sales Order Item,WH003,-520.0,5800,3180.0,2680.0,1600
4,2025-01-04,SO005-2,Sales Order Item,WH002,-870.0,5800,2310.0,2680.0,1600
5,2025-01-04,SO006-1,Sales Order Item,WH001,-50.0,5750.0,2310.0,2680.0,1600
6,2025-01-10,SO009-1,Sales Order Item,WH001,-650.0,5100.0,2310.0,2680.0,1600
7,2025-01-14,PO020-2,Purchase Order Item,WH001,800.0,5900.0,2310.0,2680.0,1600
8,2025-01-21,PO032-1,Purchase Order Item,WH002,640.0,5900.0,2950.0,2680.0,1600
9,2025-01-21,SO024-1,Sales Order Item,WH002,-770.0,5900.0,2180.0,2680.0,1600
